# Script used to generate eQTL plots with the COLOC Bayesian posterior probability estimate

1. extract the gene of interest from the tissue of interest from the GTEx.allpairs.txt.gz files (check the SNP within the GTEx website for tissue of interest) - function with get_info_gtex 
2. Run process_ref() function to extract the rsids and allele information from part 1
3. firstmerge() to merge this information together
4. plink command - run_plink() to generate the LD estimates for each snp
5. get the alleles to be in the right direction - flip()
6. make the graph make_graph() 
7. Make smaller df with just SNP, beta and SE for each trait, then calculate the bayes factor make_bayes_factors() and then combined the both the datasets combine_df()
8. Then calculate the PP by -> print (get_PP(combined, "rs12441817")) 

In [7]:
import numpy as np
import math
import sumstats
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def get_infor_gtex(RSID, GENE_NAME , TISSUE_NAME, GTEX_FILE_NAME, ENSEMBLE):
    '''
    This is a hackme function as I have not written a function to process the 20GB GTEx files
    ENSEMBLE is the ID for the gene of interest (not the isoform - I guess it would work)
    GTEX_FILE_NAME is the *.all_pairs.txt.gz for a given tissue of interest
    GENE_NAME and TISSUE_NAME are used to name the grep output file
    '''
    
    file_name_lung = RSID + ".find."+GENE_NAME+ "_" + TISSUE_NAME + ".txt"
    command1 = "zgrep " + str(ENSEMBLE)  + " " + GTEX_FILE_NAME + " > " + file_name_lung
    os.system(command1)


def make_bayes_factors(df):
    """
    returns a df with just the SNP and the bayes factor for each SNP
    INPUT DF contains SNP, beta, se
    """ 
    W = 0.2 
    df["Z"] = df["beta"] / df["se"]
    df["V"] = df["se"] ** 2 
    df["R"] =  W **2 / (W**2 + df["V"])
    df["lbf"] =  0.5 * (np.log(1 - df["R"]) + (df["R"] * df["Z"]**2))
    df_small = df[["SNP", "lbf" , "Z"]].copy()
    return df_small 


def get_PP4(
    title: str,
    pp0: float,
    pp1: float,
    pp2: float,
    pp3: float,
    pp4: float):
    """part of calling the below function"""
    return pp4


def coloc1(trait1_lnbfs, trait2_lnbfs, prior1: float = 1e-4, prior2: float = 1e-4, prior12: float = 1e-5 ):
    """
    COLOC function taken from Anthony Aylward
    https://github.com/anthony-aylward/coloc
    """
    log_numerators = (
        0,
        math.log(prior1) + sumstats.log_sum(trait1_lnbfs),
        math.log(prior2) + sumstats.log_sum(trait2_lnbfs),
        math.log(prior1) + math.log(prior2) + sumstats.log_sum(
            trait1_lnbf + trait2_lnbf
            for i, trait1_lnbf in enumerate(trait1_lnbfs)
            for j, trait2_lnbf in enumerate(trait2_lnbfs)
            if i != j
        ),
        math.log(prior12) + sumstats.log_sum(
            trait1_lnbf + trait2_lnbf
            for trait1_lnbf, trait2_lnbf in zip(trait1_lnbfs, trait2_lnbfs)
        )
    )
    yield from  (
        math.exp(log_numerator - sumstats.log_sum(log_numerators))
        for log_numerator in log_numerators
    )

def get_PP(df, snp):
    '''
    This is to do the PP for the coloc
    '''
    PP4 = get_PP4("snp", *coloc1((df.GWAS_1_lbf.to_numpy()),(df.GWAS_2_lbf.to_numpy() )))
    PP4_rounded = round(PP4,4)
    raw = [[ snp , PP4_rounded]]
    df2 = pd.DataFrame(data=raw, columns=["SNP", "PP4"])
    return df2


def combine_df(df1 , df2):
    """
    merged the 2 smaller DFs together, just incase SNPs are missing form one, it will push out the position each snp in the np array
    """
    df1.columns = ["SNP", "GWAS_1_lbf", "GWAS_1_Z"]
    df2.columns = ["SNP", "GWAS_2_lbf", "GWAS_2_Z"]
    merged = df1.merge(df2,on="SNP")
    return merged

import os 

def run_plink(DF, RSID): 
    lung_SNP_list = DF[["rsid"]].copy()
    lung_SNP_list.to_csv("for_plink_r2.txt", sep="\t",index=None)
    lung_plink1 = "plink --bfile ../PRS_work/LC_PRS --extract for_plink_r2.txt --make-bed --out temp"
    lung_plink2 = "plink --bfile temp --ld-snp "+ RSID + " --ld-window 10000 --ld-window-kb 10000 --ld-window-r2 0 --out" +RSID+" --r2 "
    os.system(lung_plink1)
    os.system(lung_plink2)
    temp = add_rs(DF,RSID)
    return temp

def add_rs(df, RSID):
    ld_file = RSID + ".ld"
    current_snp = pd.read_csv(ld_file, delimiter=r"\s+")
    current_snp = current_snp[["SNP_B", "R2"]].copy()
    current_snp.columns=["rsid", "scale"]
    temp = df.merge(current_snp , on="rsid")
    return temp

def process_gtex_file(file): 
    temp = pd.read_csv(file, sep="\t", header=None)
    temp.columns = ['gene_id', 'variant_id', 'tss_distance', 'ma_samples', 'ma_count','maf', 'pval_nominal', 'slope', 'slope_se']
    temp["gtex_Z"] = round((temp.slope / temp.slope_se),4) 
    
    temp = temp.drop(['tss_distance', 'ma_samples', 'ma_count','maf', 'pval_nominal' ],1)
    return temp
    
def process_ref(chrom):
    temp = pd.read_csv("ref_GTEX_hg38_chr"+str(chrom)+".txt" , sep="\t") 
    temp = temp[["variant_id", "alt", "rs_id_dbSNP151_GRCh38p7"]].copy()
    temp.columns =["variant_id", "gtex_alt", "rsid"]
    return temp
                     

def first_merge(data_df ,gtex_df, ref_df):
    temp = gtex_df.merge(ref_df, on="variant_id")
    temp2 = temp.merge(data_df, left_on="rsid", right_on="UKFH_SNP")
    smaller1 = temp2[["rsid", "gtex_alt", "gtex_Z" , "UKFH_ref_old", "UKFH_alt_old", "META_beta_UKFH", "META_STD_FE", 'slope', 'slope_se']].copy()
    smaller1["META_ZLC_raw"] = round((smaller1.META_beta_UKFH / smaller1.META_STD_FE),4) 
    return smaller1

def flip(df): 
    needs_flipping = df[df["gtex_alt"] != df["UKFH_alt_old"]]
    cob_clean = df[~df['rsid'].isin(needs_flipping["rsid"])]
    needs_flipping["META_ZLC"] = needs_flipping.META_ZLC_raw * -1 
    cob_clean["META_ZLC"] = cob_clean.META_ZLC_raw
    combined = pd.concat([cob_clean, needs_flipping])
    return combined

def SetColor(x):
    if(float(x) < 0.1):
        return "#E0E0E0"
    elif(float(x) >= 0.1) & (float(x) <= 0.4):
        return "#9E9E9E"
    elif(float(x) >= 0.4) & (float(x) <= 0.8):
        return "yellow"
    elif(float(x) > 0.8) &  (float(x) < 0.9999):
        return "rgb(227,26,28)"
    elif(float(x) > 0.9999):
        return "purple"

def make_graph(df, tissue , gene, rsid): 
    final1 = df.sort_values("scale",ascending=True)
    final1['marker_size'] = (final1['scale'] + 1 ) * 9
    combined2 = final1[final1["scale"] != 1 ]
    combined3 = final1[final1["scale"] == 1 ]
    Y_name = "<b> "+ gene + " " + tissue + " expression t-score </b>"
    title_plot = rsid + " in " + gene
    import plotly.graph_objects as go
    import plotly
    fig = go.Figure()
    fig.update_layout(yaxis=dict(range=[-6,6]))
    fig.update_layout(xaxis=dict(range=[-6,6]))
    fig.update_layout(plot_bgcolor='white' ) 
    fig.update_layout(title={'text': title_plot,'x':0.5 , 'xanchor': 'center','yanchor': 'top' , 'font_color':"black"})
    fig.update_xaxes(ticks="outside", title="<b> Lung Cancer Meta Z-stat </b>"  ,zeroline = True, zerolinewidth = 3, zerolinecolor="black"  ,showgrid = False,  gridwidth = 1,gridcolor = '#bdbdbd', color='black', tickwidth=5, tickcolor='black', ticklen=10, showline=True, linewidth=5, linecolor='Black')
    fig.update_yaxes(ticks="outside", title=Y_name , zeroline = True, zerolinewidth = 3,zerolinecolor="black", showgrid = False, gridwidth = 1,gridcolor = '#bdbdbd',color='black',tickwidth=5, tickcolor='black', ticklen=10, showline=True, linewidth=5, linecolor='Black')
    fig.update_layout(font=dict(size=20) ) 
    fig.add_trace(go.Scatter( name="R-square", mode="markers"  , x = combined2["META_ZLC"].tolist(), y = combined2["gtex_Z"].tolist(), 
    showlegend=False , marker_size=combined2['marker_size'],opacity=1, marker_line_width=0.8,line=dict(color="black"), marker_color=list(map(SetColor, combined2['scale']))))
    fig.add_trace(go.Scatter( name="SNP", text=combined3["rsid"], mode="markers" ,marker_symbol="asterisk" , x = combined3["META_ZLC"].tolist(), y = combined3["gtex_Z"].tolist(),
    showlegend=False , marker_size=20,opacity=1, marker_line_width=3,line=dict(color="black"), marker_color=list(map(SetColor, combined3['scale']))))   
    output_name = rsid + "_" + gene + "_" + tissue + ".png"
    fig.write_image(output_name,width=1000, height=800, scale=10)
    fig.show()


In [8]:
data = pd.read_csv("~/all_snps_UKFH_orientation.txt" , sep="\t") 
CYP1A1 = process_gtex_file("rs12441817.find.CYP1A1_Brain_Nucleus_accumbens.txt")
ref_15 = process_ref(15)
clean = first_merge(data,CYP1A1,ref_15)
clean = run_plink(clean, "rs12441817")
clean1 = flip(clean)
make_graph(clean1, "Nucleus_accumbens" , "CYP1A1" , "rs12441817" )
test1 = clean[["rsid", "META_beta_UKFH", "META_STD_FE"]].copy()
test1.columns = ["SNP", "beta" , "se" ]
test2 = clean[["rsid" , "slope", "slope_se"]].copy()
test2.columns = ["SNP", "beta" , "se" ]
moo = make_bayes_factors(test1)
moo2 = make_bayes_factors(test2)
combined = combine_df(moo,moo2)
combined = combined.dropna()
print (get_PP(combined, "rs12441817"))

          SNP     PP4
0  rs12441817  0.7026
